## Extraindo dados a partir de CSVs

### Caso de uso: Dados do projeto Konsulta

<img src="imgs/pandas-csv.png" width="600">


In [15]:
import pandas as pd
import ast
from datetime import datetime


In [16]:

dfTimeline = pd.read_csv('dataset_konsulta/timelines.csv')
dfTimeline.shape

(5645, 5)

In [17]:
dfTimeline.head()

,_id,ov,clientName,createdAt,status
0,5fdd01899591691c943877ab,1304464,MOINHO TAQUARIENSE LTDA,1608319369639,"[{""step"":1,""label"":""Ov encontrada"",""content"":{..."
1,5fe09ee7c5f47351cef72e51,1304465,MINERAÇÃO BELOCAL LTDA,1608556263663,"[{""step"":1,""label"":""Ov encontrada"",""content"":{..."
2,5fe0c9e0c5f47351ce27b710,6470722,YPIOCA INDUSTRIAL DE BEBIDAS S.A,1608567264022,"[{""step"":1,""label"":""Ov encontrada"",""content"":{..."
3,5fe24a3bc5f47351cec79031,1304194,CP KELCO BRASIL S/A,1608666097633,"[{""step"":1,""label"":""Ov encontrada"",""content"":{..."
4,5fe328efc5f47351cead6e3b,1304415,GLOBAL PAPEIS EIRELI,1608722671882,"[{""step"":1,""label"":""Ov encontrada"",""content"":{..."


In [18]:
dfTimeline['statusFinal'] = dfTimeline["status"].astype('str')
dfTimeline['statusFinal'] = dfTimeline['statusFinal'].apply(lambda x: ast.literal_eval(x))
dfTimeline['statusFinal'] = dfTimeline['statusFinal'].apply(lambda x: x[-1])
lst_liberado = ['Status do Pedido: Operação o.k.','Status do Pedido: Operação liberada', 'Ov liberada', 'Liberado.', 'Problema ao liberar pedido. Favor verificar se já está liberado', 'Operação liberada', 'Crédito dentro do limite']
lst_bloqueado = ['Status do Pedido: Operação não o.k.', 'Operação não o.k.', 'Status do Pedido: não executada']
# df['createdAt'] = pd.to_datetime(df['createdAt'], errors='coerce')


dfTimeline.shape

(5645, 6)

In [19]:
def buscaString(x):
    if x['label'].strip() in lst_liberado:
        return 'Liberado'
    elif 'Pedido bloqueado' in x['label'] or x['label'].strip() in lst_bloqueado :
        return 'Bloqueado'
    else:
        return 'Sem status'

In [20]:
dfTimeline['finalizada'] = dfTimeline['statusFinal'].apply(lambda x: buscaString(x))
dfTimeline.rename(columns={ 'createdAt_y' : 'createdAt'})
dfTimeline.head()

,_id,ov,clientName,createdAt,status,statusFinal,finalizada
0,5fdd01899591691c943877ab,1304464,MOINHO TAQUARIENSE LTDA,1608319369639,"[{""step"":1,""label"":""Ov encontrada"",""content"":{...","{'step': 2, 'label': 'Pedido bloqueado - Fatur...",Bloqueado
1,5fe09ee7c5f47351cef72e51,1304465,MINERAÇÃO BELOCAL LTDA,1608556263663,"[{""step"":1,""label"":""Ov encontrada"",""content"":{...","{'step': 2, 'label': 'Pedido bloqueado - Fatur...",Bloqueado
2,5fe0c9e0c5f47351ce27b710,6470722,YPIOCA INDUSTRIAL DE BEBIDAS S.A,1608567264022,"[{""step"":1,""label"":""Ov encontrada"",""content"":{...","{'step': 2, 'label': 'Pedido bloqueado - Fatur...",Bloqueado
3,5fe24a3bc5f47351cec79031,1304194,CP KELCO BRASIL S/A,1608666097633,"[{""step"":1,""label"":""Ov encontrada"",""content"":{...","{'step': 3, 'label': 'Pedido bloqueado - Esse ...",Bloqueado
4,5fe328efc5f47351cead6e3b,1304415,GLOBAL PAPEIS EIRELI,1608722671882,"[{""step"":1,""label"":""Ov encontrada"",""content"":{...","{'step': 2, 'label': 'Pedido bloqueado - Limit...",Bloqueado


In [21]:
dfTimeline['status'][0]

'[{"step":1,"label":"Ov encontrada","content":{"Limite de crédito":"R$ 1.000.000,00","Compromisso Total":"R$ 368.707,72","Valor Pedido":"R$ 22.832,70","Grau de utilização":"36.87%"},"icon":"search"},{"step":2,"label":"Pedido bloqueado - Faturas em Atraso, esse pedido não será liberado até que as faturas sejam regularizadas","icon":"slash"}]'

<img src="imgs/lambda.png" width="600">


### Biblioteca Python ast – Abstract Syntax Trees

Transformar strings em objetos Python com literal_eval


In [22]:
dfTimeline['status'] = dfTimeline['status'].apply(lambda x: ast.literal_eval(x))

In [23]:
dfTimeline['horaUltimoStatus'] = pd.to_datetime(dfTimeline['createdAt'],unit='ms').dt.strftime('%d/%m/%Y %H:%M')


### Boolean Indexing

Aplicando filtro ou criterios condicionais para verificações boleanas

In [24]:
#remove os "Sem status"

dfTimeline = dfTimeline[dfTimeline.finalizada != 'Sem status']
dfTimeline.shape

(3959, 8)

In [25]:
periodoFinal = pd.to_datetime(dfTimeline['createdAt'],unit='ms').max().strftime('%d/%m/%Y')
periodoInicial = pd.to_datetime(dfTimeline['createdAt'],unit='ms').min().strftime('%d/%m/%Y')
print("Dados de ", periodoInicial, " até ", periodoFinal)

Dados de  18/12/2020  até  17/03/2021


In [26]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(labels=sorted(dfTimeline['finalizada'].unique()), 
                            values=dfTimeline['finalizada'].value_counts().sort_values())])
colors = ['tomato','#32c691']

fig.update_traces(marker=dict(colors=colors))


fig.update_traces(hoverinfo='label+percent', textinfo='value')
fig.update_layout(title_text="Status dos pedidos entre " + str(periodoInicial) + " até " + str(periodoFinal), title_font_size=20)


fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(
    margin=dict(l=20, r=20, t=100, b=20),
)
fig.update_layout(
    annotations=[dict(text='Total = '+str(len(dfTimeline['finalizada']))+' pedidos', x=1.15, y=-0.003, font_size=20, showarrow=False)])
fig.show()


In [27]:
dfOvsLiberadas = dfTimeline.reset_index(drop=True)
dfOvsLiberadas.head()

,_id,ov,clientName,createdAt,status,statusFinal,finalizada,horaUltimoStatus
0,5fdd01899591691c943877ab,1304464,MOINHO TAQUARIENSE LTDA,1608319369639,"[{'step': 1, 'label': 'Ov encontrada', 'conten...","{'step': 2, 'label': 'Pedido bloqueado - Fatur...",Bloqueado,18/12/2020 19:22
1,5fe09ee7c5f47351cef72e51,1304465,MINERAÇÃO BELOCAL LTDA,1608556263663,"[{'step': 1, 'label': 'Ov encontrada', 'conten...","{'step': 2, 'label': 'Pedido bloqueado - Fatur...",Bloqueado,21/12/2020 13:11
2,5fe0c9e0c5f47351ce27b710,6470722,YPIOCA INDUSTRIAL DE BEBIDAS S.A,1608567264022,"[{'step': 1, 'label': 'Ov encontrada', 'conten...","{'step': 2, 'label': 'Pedido bloqueado - Fatur...",Bloqueado,21/12/2020 16:14
3,5fe24a3bc5f47351cec79031,1304194,CP KELCO BRASIL S/A,1608666097633,"[{'step': 1, 'label': 'Ov encontrada', 'conten...","{'step': 3, 'label': 'Pedido bloqueado - Esse ...",Bloqueado,22/12/2020 19:41
4,5fe328efc5f47351cead6e3b,1304415,GLOBAL PAPEIS EIRELI,1608722671882,"[{'step': 1, 'label': 'Ov encontrada', 'conten...","{'step': 2, 'label': 'Pedido bloqueado - Limit...",Bloqueado,23/12/2020 11:24


In [28]:
dfOvsLiberadas[['ov', 'clientName', 'finalizada', 'horaUltimoStatus']].to_excel("OVS_e_status.xlsx")